In [ ]:
from diffusers import StableDiffusionXLPipeline, EulerAncestralDiscreteScheduler
import torch

model_id = "aipicasso/emi"

scheduler = EulerAncestralDiscreteScheduler.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    
    subfolder="scheduler",
)
pipe = StableDiffusionXLPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
pipe.enable_freeu(b1=1.1, b2=1.2, s1=0.6, s2=0.4)


In [ ]:
# 職業
jobs = sum([
    ["knight"]   * 20,
    ["noble"]    * 20,
    ["swordman"] * 10,
    ["archer"]   * 5,
    ["warrior"]  * 20,
    ["thief"]    * 1,
    ["priest"]   * 1,
    ["mage"]     * 1,
    ["merchant"] * 1,
    ["pirate"]   * 1,
    ["NONE"]     * 10,
], [])


# 性別
jenders = sum([
    ["1boy"]                * 10,
    ["1boy, (shota)"]         * 10,
    ["1boy, teenage"]       * 10,
    ["1boy, adult"]         * 10,
    ["1girl"]               * 10,
    ["1girl, (loli)"]         * 10,
    ["1girl, teenage"]      * 10,
    ["1girl, adult"]        * 10,
    ["solo, mature male"]   * 5,
    ["solo, mature female"] * 5,
    ["solo, old man"]       * 3,
    ["solo, old woman"]     * 3,
    ["solo, facial hair"]   * 3,
], [])


hair_colors = sum([
    ["black hair"]  * 5,
    ["blond hair"]  * 5,
    ["brown hair"]  * 5,
    ["red hair"]    * 4,
    ["green hair"]  * 4,
    ["blue hair"]   * 4,
    ["silver hair"] * 1,
    ["purple hair"] * 1,
    ["pink hair"]   * 1,
    ["NONE"]        * 1,
], [])

# 髪型
hair_types = sum([
    ["long hair"]  * 1,
    ["short hair"] * 1,
    ["NONE"]       * 8,
], [])


# 装飾
ornaments = sum([
    ["hair ornament"] * 1,
    ["ribbon"]        * 1,
    # ["glasses"]       * 1,
    ["scarf"]         * 1,
    # ["gloves"]        * 1,
    ["sword"]         * 1,
    ["bow"]           * 1,
    # ["eyepatch"]      * 1,
    ["scar"]          * 1,
    ["NONE"]          * 10,
], [])

# 性格
personalities = sum([ 
    ["calm"]        * 2,
    ["brave"]       * 5,
    ["intelligent"] * 1,
    ["smile"]       * 1,
    ["charming"]    * 1,
    ["strong"]      * 1,
    ["serious"]     * 1,
    ["generous"]    * 1,
    ["bright"]      * 1,
    ["vivid"]       * 1,
    ["NONE"]        * 0,
], [])



In [ ]:
import random
def pick_one(arr):
    one = random.choice(arr)
    return one

def generate_prompt():
    randoms = [
        pick_one(jobs),
        pick_one(jenders),
        pick_one(hair_colors),
        pick_one(hair_types),
        pick_one(ornaments),
        pick_one(personalities),
    ]
    randoms = [r for r in randoms if r != "NONE"]
    random_prompt = ", ".join(randoms)
    # prompt = f"anime artwork, anime style, (portrait), {random_prompt}, dark skin, medieval, fantasy, ((transparent))"
    prompt = f"anime artwork, anime style, (portrait), {random_prompt}, medieval, fantasy, ((transparent))"
    return prompt

from pathlib import Path
out_dir = Path("outputs")
out_dir.mkdir(exist_ok=True)
image_id = 0

from tqdm.auto import tqdm

for i in tqdm(range(10000)):
    prompt = generate_prompt()
    filename = f"{prompt}__{image_id:05d}.png"
    out_path = out_dir / filename
    image_id += 1

    nega = "(embedding:unaestheticXLv31:0.5), photo, deformed, realism, disfigured, low contrast, bad hand"
    image = pipe(
        prompt=prompt,
        negative_prompt=nega,
        num_inference_steps=20,
    ).images[0]
    image.save(out_path)
